In [51]:
import numpy as np
from sympy import *
from matplotlib import pyplot as plt

In [52]:
def Hadamard(i):
    global N_qubit
    H=np.identity(2*N_qubit)
    H[i,i+N_qubit]=-1
    H[i+N_qubit,i]=1
    H[i,i]=0.
    H[i+N_qubit,i+N_qubit]=0.
    
    return H
    

In [53]:
def x_parity(i,j):
    global N_qubit
    x_par=Hadamard(i)@SUM(i,j)@np.linalg.inv(Hadamard(i))
    
    return x_par

In [54]:
def SUM(i,j):
    global N_qubit
    Sum_ij=np.identity(2*N_qubit)
    Sum_ij[i+N_qubit,j+N_qubit]=-1.0
    Sum_ij[j,i]=1.0
    
    return Sum_ij

In [55]:
def inv_SUM(i,j):
    return np.linalg.inv(SUM(i,j))

In [56]:
def CZ(i,j):
    global N_qubit
    CZ=Hadamard(j)@SUM(i,j)@np.linalg.inv(Hadamard(j))
    
    return CZ

In [57]:
def inv_CZ(i,j):
    return np.linalg.inv(CZ(i,j))

In [58]:
def logical_err(q,p):
    q_err=np.abs(Rs(2*np.sqrt(np.pi),q))
    p_err=np.abs(Rs(2*np.sqrt(np.pi),p))
    
    if q_err>0.5*np.sqrt(np.pi) or p_err>0.5*np.sqrt(np.pi):
        logic=1.0
    else:
        logic=0.
        
    return logic

In [59]:
def Rs(s,z):
    return z-s*np.floor(z/s+1/2)

## Initialize error vector

In [60]:
q1,q2,q3,q4,q5,p1,p2,p3,p4,p5 = symbols('q1 q2 q3 q4 q5 p1 p2 p3 p4 p5')

In [61]:
N_qubit=5

In [62]:
B = Matrix([q1, q2, q3, q4, q5, p1, p2, p3, p4, p5])

## First layer of encoder

In [63]:
#B1=(SUM(0,2)@SUM(0,1))*B
A1=SUM(0,1)

In [64]:
print(A1.shape)

(10, 10)


## Second layer of encoder

In [65]:
#B2=Hadamard(0)*B1
A2=SUM(0,2)@A1

In [66]:
A3=SUM(0,3)@A2
A4=SUM(0,4)@A3

In [67]:
dec_cir=np.linalg.inv(A4)

# decoder is the inverse of encoder A4

In [68]:
print(dec_cir)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


In [69]:
# C-matrix in Eq.30
C=np.transpose(np.linalg.inv(dec_cir))@np.linalg.inv(dec_cir)


In [70]:
# calculate the estimated zq, zp. Then perform error correction
z_q=np.zeros(2*N_qubit)
z_p=np.zeros(2*N_qubit)
for i in np.arange(2*N_qubit):
    z_q+=-1/C[0,0]*C[0,i]*dec_cir[i]
    z_p+=-1/C[N_qubit,N_qubit]*C[N_qubit,i]*dec_cir[i]

    
z_q+=1/C[0,0]*C[0,0]*dec_cir[0]
z_p+=1/C[N_qubit,N_qubit]*C[N_qubit,N_qubit]*dec_cir[N_qubit]
    
q_corrected=dec_cir[0]-z_q
p_corrected=dec_cir[N_qubit]-z_p

In [71]:
# Calculate variance after EC
print(np.sum(q_corrected**2))
print(np.sum(p_corrected**2))

0.20000000000000007
1.0


In [73]:
A_mtx=np.delete(dec_cir,0, axis=0)
A_mtx=np.delete(A_mtx,N_qubit-1, axis=0)

print(A_mtx)

[[-1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


In [74]:
mtx=dec_cir@(np.identity(10)-np.transpose(A_mtx)@np.linalg.inv(A_mtx@np.transpose(A_mtx))@A_mtx)

In [76]:
times=10**7

#corrected_state=np.zeros((times,6))

err_sweep=np.linspace(0.1,0.4,31)

logical_rate=np.zeros(err_sweep.shape[0])

In [77]:
for j in range(err_sweep.shape[0]):
    print(j)
    sigma=err_sweep[j]
    
    for i in range(times):
        s=np.random.normal(0.0,sigma,(10))

        syn=Rs(np.sqrt(2*np.pi),A_mtx@s)
        corrected_state=dec_cir@s-dec_cir@np.transpose(A_mtx)@np.linalg.inv(A_mtx@np.transpose(A_mtx))@syn
        q_out=corrected_state[0]
        p_out=corrected_state[5]
        
        logical_rate[j]+=1/times*logical_err(q_out,p_out)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
